In [1]:
!pip install nltk

In [2]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
import random
import re
import requests
import json
import tkinter as tk
from tkinter import *
from tkinter import messagebox, simpledialog


def blackout(text):
    global blackout_text
    blackout_text = ""
    
    for i, char in enumerate(text):
        in_range = False
        for (start, end) in result_text_index:
            if i >= start and i <= end:
                in_range = True
                break
        if in_range:
            blackout_text += char

        elif char == "\n":
            blackout_text += "\n"
        elif char == " ":
            blackout_text += " "
        else:
            blackout_text += "▀"
            
            
def search_wiki(user_input):
        global text, message, ambig
        ambig = False

        try:
            try:
                ambig = False
                message = ""
                text = wiki_pull(user_input)
                if text == "":
                    text = wiki_pull_backup(user_input)
                    message = f"Your search terms weren't a perfect match. Here's the closest article we found: {closest_match}\n"
                return text, ambig
                    
            except Exception as e:
#                 print(f"An error occurred: {e}")
                text = wiki_pull_backup(user_input)
                if "may refer to" not in text:
                    return text, ambig
                    
                ambig = True
                return None, ambig

                
        except Exception as e:
#             print(f"An error occurred: {e}")
            message = f"\nYour search terms didn't match any results! Please try again"
            update_poem()



def wiki_pull(user_input):
    wiki_api_url = "https://en.wikipedia.org/w/api.php"

    params = {
        "action": "query",
        "format": "json",
        "prop": "extracts",
        "exintro": "", # get intro section only
        "explaintext": "", # plain text
        "titles": user_input}

    response = requests.get(wiki_api_url, params=params)
    response_json = response.json()

    page_id = list(response_json["query"]["pages"].keys())[0]
    page_content = response_json["query"]["pages"][page_id]["extract"]

    # Check if page is a may-refer-to disambiguation page
    if 'may refer to' in page_content or 'disambiguation' in page_content:
        message = f"\nThe page for {user_input} is a disambiguation page. Please try again with a more specific search term."
        raise ValueError(f"The page for {user_input} is a disambiguation page.")
        
    return page_content





def wiki_pull_backup(user_topic):
    
    wiki_api_url = "https://en.wikipedia.org/w/api.php"

    # using search action instead of query
    params = {
        "action": "query",
        "format": "json",
        "list": "search",
        "srsearch": user_topic,
        "srprop": "",
        "utf8": ""}

    # making request
    response = requests.get(wiki_api_url, params=params)
    response_json = response.json()

    # selecting the page with closest match--IGNORING DISAMBIGUATION PAGES
    global closest_match
    for result in response_json["query"]["search"]:
        if "disambiguation" not in result["title"].lower(): # check if snippet contains "disambiguation"
            closest_match = result["title"]
            break

    # using the selected title for content request
    params = {
        "action": "query",
        "format": "json",
        "prop": "extracts",
        "exintro": "", # get intro section only
        "explaintext": "", # plain text
        "titles": closest_match}

    # making request for content
    response = requests.get(wiki_api_url, params=params)
    response_json = response.json()

    # have to get page_id so we can index the json dictionary for the extract
    page_id = list(response_json["query"]["pages"].keys())[0]
    page_content = response_json["query"]["pages"][page_id]["extract"]

    return page_content







# the parts of speech that are allowed in the output
allowed_pos = ["JJ", "RB", "VB", "NN", "IN"]


result_tuples = []
chosen = []

# pops is used to limit the number of loops
max_pops = 25
global pops
pops = 0




def text2tokens(text):
    global tokens, text_copy
    tokens = word_tokenize(text)
    
    # to preserve a version of the original text
    text_copy = text
        
    # attaching the index from the tokenized text AND the original text to each word (both are later added to result_tuple)
    global both_indices
    both_indices = []
    for i, token in enumerate(tokens):
        text_index = text_copy.find(token) # searches for substring in string
        text_index_end = text_index + (len(token)-1)
        text_copy = text_copy.replace(token, "x" * len(token), 1) # replace the current token with all x's
        both_indices.append((token, i, text_index, text_index_end))



def get_word():
    global choice, new_word, new_word_index, new_word_pos
    choice = random.choice(range(len(tokens)))
    chosen.append(choice)
    new_word = tokens[choice]
    new_word_index = choice
    new_word_pos = pos_tag([new_word])[0][1][:2] # syntax: pos_tag generates a list of a tuple. This grabs the tuple, then grabs the 2nd element which is the POS as a string, then grabs the first 2 letters.
    
    
    
def sequence_check():
    """Checks to see if there is an sequence like 'adjective then noun' around the chosen word. 
    If so, then it adds both words to the results list."""
    
    global new_word, new_word_index, new_word_pos
    if choice > 0 and choice < len(tokens) - 1: # avoids list index out of range errors
        prior_word = tokens[choice-1]
        prior_word_index = choice-1
        prior_word_pos = pos_tag([prior_word])[0][1][:2]

        next_word = tokens[choice+1]
        next_word_index = choice+1
        next_word_pos = pos_tag([next_word])[0][1][:2]
    
    # adj-noun-verb
        if prior_word_pos == "JJ" and new_word_pos == "NN" and next_word_pos =="VB":
            new_word = prior_word
            new_word_index = prior_word_index
            new_word_pos = prior_word_pos
            check_and_insert()
            new_word = next_word
            new_word_index = next_word_index
            new_word_pos = next_word_pos
            check_and_insert()
#             print("adj-noun-verb worked :)")
    
    # adj-noun
        elif prior_word_pos == "JJ" and new_word_pos == "NN":
            new_word = prior_word
            new_word_index = prior_word_index
            new_word_pos = prior_word_pos
            check_and_insert()
#             print("adj-noun worked :)")

        elif new_word_pos == "JJ" and next_word_pos == "NN":
            new_word = next_word
            new_word_index = next_word_index
            new_word_pos = next_word_pos
            check_and_insert()
#             print("adj-noun worked :)")
     #modal-verb
        elif prior_word_pos == "MD" and new_word_pos =="VB":
            new_word = next_word
            new_word_index = prior_word_index
            new_word_pos = prior_word_pos
            check_and_insert()         
        
    #Determiner and noun
        elif new_word_pos == "DT" and next_word_pos =="NN":
            new_word = next_word
            new_word_index = next_word_index
            new_word_pos = next_word_pos
            check_and_insert()
                
    #Predeterminer and noun
        elif new_word_pos == "PDT" and next_word_pos =="NN":
            new_word = next_word
            new_word_index = next_word_index
            new_word_pos = next_word_pos
            check_and_insert()
    #Preposition and noun
        elif new_word_pos == "IN" and next_word_pos =="NN":
            new_word = next_word
            new_word_index = next_word_index
            new_word_pos = next_word_pos
            check_and_insert()
    
    
    
    
def check_and_insert():
    # Criteria:

    # regular expression that matches only word characters. Prevents stuff like "]" which is a noun for some reason.
    pattern = re.compile(r'\w+')
    if pattern.match(new_word):
    
    
    
        if new_word_pos in allowed_pos:

            # find the index to insert the new word at.
            index_to_insert = len(result_tuples)
            for i, (word, index, pos, text_index, text_index_end) in enumerate(result_tuples): # this uses tuple unpacking and the enumerate function
                if new_word_index < index:
                    index_to_insert = i
                    break



            # check if the new_word_pos is different from the POS of the words before and after the insertion point. checks 2 things:
                # 1. does the word before the insertion point have a different POS, or is the new word gonna be the first word in the list.
                # 2. does the word after the insertion point have a different POS, or is the new word gonna be the last word in the list.
            if (index_to_insert == 0 or result_tuples[index_to_insert-1][2] != new_word_pos) and (index_to_insert == len(result_tuples) or result_tuples[index_to_insert][2] != new_word_pos):
                
                # getting index from original text
                for (token, token_index, text_index, text_index_end) in both_indices:
                    if new_word_index == token_index:
                        new_text_index = text_index
                        new_text_index_end = text_index_end
                
                # insert the new word's tuple into the result_tuples list
                result_tuples.insert(index_to_insert, (new_word, new_word_index, new_word_pos, new_text_index, new_text_index_end))


        if len(result_tuples) == random_length:


            # force the first word to not be a verb
            first_word_pos = result_tuples[0][2][:2]
            if first_word_pos == "VB":
                result_tuples.pop()
                global pops
                pops +=1
    


def display_results():
    global result_words, result_index, result_pos, result_text_index
    result_words = []
    result_index = [] # result_index and result_pos lists are for easy reference to ensure the criteria worked
    result_pos = []
    result_text_index = []
    for (word, index, pos, text_index, text_index_end) in result_tuples:
        result_words.append(word)
        result_index.append(index)
        result_pos.append(pos)
        result_text_index.append((text_index, text_index_end))


        



while_counter = 0
def update_poem():
    user_input = search_entry.get()

    text, ambig = search_wiki(user_input)
    if ambig:
        poem = f"\nThe page for {user_input} is a disambiguation page. Please try again with a more specific search term."
        poemtext.insert(INSERT,poem)
        poemtext.config(state = DISABLED)
        scrolly = Scrollbar(poemtext)
        poemtext.pack(anchor= 'center', expand = True)
        return None
    
    text2tokens(text)
    
    global random_length, tokens
    random_length = random.choice(range(7,round(len(tokens)/3)))
    
    while_counter = 0
    while len(result_tuples) < random_length and pops < 25 and while_counter < 150:
        get_word()
        check_and_insert()
        sequence_check()
        
        while_counter +=1
    

    display_results()
    blackout(text)
    
    global result_string
    delimiter = " "
    result_string = delimiter.join(result_words)
    
    poem = f"{message}\n\n{text}"
#     poem = f"{message}\n{len(result_words)}\n{result_string}\n\n{text}\n\n{blackout_text}\n"
    
    poemtext.insert(INSERT,poem)
    poemtext.config(state = DISABLED)
    scrolly = Scrollbar(poemtext)
    poemtext.pack(anchor= 'center', expand = True)


    
    
    
def refresh():
    global poemtext
    user_input = search_entry.get()
    
    
    pops = 0
    
    poemtext.destroy()
    poemtext = Text(root, wrap = "word", spacing1 = 1, spacing2 = 2, spacing3 = 1, width = 100, height = 100, font=("Verdana", 16))  

    if ambig:
        poem = f"\nThe page for {user_input} is a disambiguation page. Please try again with a more specific search term."
        poemtext.insert(INSERT,poem)
        poemtext.config(state = DISABLED)
        scrolly = Scrollbar(poemtext)
        poemtext.pack(anchor= 'center', expand = True)
#         return None
    
    
    global result_tuples
    result_tuples = []
    
    global random_length, tokens
    random_length = random.choice(range(7,round(len(tokens)/3)))
    
    while_counter = 0
    while len(result_tuples) < random_length and pops < 25 and while_counter < 150:
        get_word()
        check_and_insert()
        sequence_check()
        
        while_counter +=1
   
    
    display_results()
    blackout(text)
    
    global result_string
    delimiter = " "
    result_string = delimiter.join(result_words)
    
    poem = f"{message}\n\n{text}"
    
    poemtext.insert(INSERT,poem)
    poemtext.config(state = DISABLED)
    scrolly = Scrollbar(poemtext)
    poemtext.pack(anchor= 'center', expand = True)
    refresh_button.config(state="normal")

    


def blackout_button():
    global poemtext
    poemtext.destroy()
    poemtext = Text(root, wrap = "word", spacing1 = 1, spacing2 = 2, spacing3 = 1, width = 100, height = 100, font=("Verdana", 16))  
    
    poem = f"{message}\n\n{blackout_text}"    
    
    poemtext.insert(INSERT,poem)
    poemtext.config(state = DISABLED)
    scrolly = Scrollbar(poemtext)
    poemtext.pack(anchor= 'center', expand = True)

    
    
    
#tkinter setup
root = tk.Tk()
root.geometry("800x650")
root.title('Blackout Poetry Generator')

    


#search section
search_frame = tk.Frame(root)
search_frame.pack(pady=10)

search_label = tk.Label(search_frame, text='Enter a search term: ')
search_label.pack(side=tk.LEFT)

search_entry = tk.Entry(search_frame, width=30)
search_entry.pack(side=tk.LEFT)

search_button = tk.Button(search_frame, text='search', command=update_poem)
search_button.pack(side=tk.LEFT, padx=10)

#poem section
poem_frame = tk.Frame(root)
poem_frame.pack(pady=10)

poemtext = Text(root, wrap = "word", spacing1 = 1, spacing2 = 2, spacing3 = 1, width = 100, height = 100, font=("Verdana", 16))  

# button to trigger the blackout function
blackout_button_tk = Button(poem_frame, text="Blackout", command=blackout_button)
blackout_button_tk.pack(pady=10)

refresh_button = tk.Button(poem_frame, text='Refresh poem', command=refresh)
refresh_button.pack()



root.mainloop()

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/rishiheggawadi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/rishiheggawadi/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/rishiheggawadi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
